In [2]:
#メモリのリセット
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [1]:
#ライブラリのimport
import pandas as pd
import numpy as np
%matplotlib inline
#from matplotlib import pyplot as pyp
import matplotlib.pyplot as plt

In [2]:
#データ準備
#元データの作り方
#所在管理MBからデータ作成
#納入日と回収日をフィルだリングする
pre_zaikodata = pd.read_csv('20231107使用_所在管理LT_納入日0828_0929_回収日8月9月.csv',encoding='shift_jis')
pre_zaikodata.head(5)

/anaconda/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,...,回収日時,発注取消日時,発注〜印刷LT,発注〜検収LT,発注〜順立装置入庫LT,発注〜順立装置出庫LT,発注〜組立LT,発注〜回収LT,更新日時,長期滞留フラグ
0,1Z23J00091508,XCGP130,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023/9/29,...,2023/9/30 7:13,< NULL >,0.14,2.31,2.59,2.75,< NULL >,2.86,2023/9/30 7:13,< NULL >
1,1Z23J00068574,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023/9/28,...,2023/9/30 7:13,< NULL >,0.15,1.99,2.23,3.79,< NULL >,3.86,2023/9/30 7:13,< NULL >
2,1Z23J00066432,XCGL630,30357,35847ECE010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,40,（株）メタルテック,< NULL >,回収済,2023/9/28,...,2023/9/30 7:13,< NULL >,0.08,2.02,2.15,3.86,< NULL >,4.02,2023/9/30 7:13,< NULL >
3,1Z23J00065428,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023/9/28,...,2023/9/30 7:13,< NULL >,0.15,1.99,2.23,3.76,< NULL >,3.86,2023/9/30 7:13,< NULL >
4,1Z23J00063618,XCGJ670,30358,35847ECE020,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,60,（株）メタルテック,< NULL >,回収済,2023/9/27,...,2023/9/30 7:13,< NULL >,2.07,3.99,4.28,6.86,< NULL >,7.00,2023/9/30 7:13,< NULL >


In [3]:
#各LTの計算

long = pre_zaikodata.iloc[:,1]
count = 0
for i in range(len(long)):
    if i % 5000==0:
            print(i)
    if ((pre_zaikodata.loc[i, '発注〜印刷LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜検収LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜回収LT'])!='< NULL >'):
        LT2 = float(pre_zaikodata.loc[i, '発注〜検収LT']) - float(pre_zaikodata.loc[i, '発注〜印刷LT'])
        LT3 = float(pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])-float(pre_zaikodata.loc[i, '発注〜検収LT'])
        LT4 = float(pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])-float(pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])
        LT5 = float(pre_zaikodata.loc[i, '発注〜回収LT'])-float(pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])
        LT6 = float(pre_zaikodata.loc[i, '発注〜回収LT'])-float(pre_zaikodata.loc[i, '発注〜検収LT'])
        pre_zaikodata.loc[i, '印刷検収LT'] = LT2
        pre_zaikodata.loc[i, '検収入庫LT'] = LT3
        pre_zaikodata.loc[i, '入庫出庫LT'] = LT4
        pre_zaikodata.loc[i, '出庫回収LT'] = LT5
        pre_zaikodata.loc[i, '社内LT（検収〜回収LT）'] = LT6
        count = count + 1
print(count)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
51857


In [5]:
###基準在庫枚数加味する
#文字型を整数型にする
#当月必要数？

pre_zaikodata['回収日時'] = pd.to_datetime(pre_zaikodata['回収日時'], errors='coerce')
pre_zaikodata['納入日（月）'] = pd.to_datetime(pre_zaikodata['納入日'], errors='coerce')

count = 0
tehaidata = pd.read_csv('20231107使用__202309_手配数_1Y_12次.csv',encoding='cp932')
#tehaidata = pd.read_csv('sample3.csv',encoding='cp932')
long2 = tehaidata.iloc[:,1]

for i in range(len(long)):
    #print(pre_zaikodata.loc[i, '回収日時'],pre_zaikodata.loc[i,'品番'] )
    if pre_zaikodata.loc[i, '納入日（月）'].month != 8:
        #print(pre_zaikodata.loc[i, '納入日（月）'].month)
        if ((pre_zaikodata.loc[i, '発注〜印刷LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜検収LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜回収LT'])!='< NULL >'):
            hinban = pre_zaikodata.loc[i,'品番'] 
            hinban2 = hinban.replace("-", "") 
            hinban3 = hinban2.replace(" ", "") 
            #print(i)
            if i % 5000==0:
                print(i)
            for j in range(len(long2)):
                #print(j)
                tehaihinban = tehaidata.loc[j,'品番'].replace('-', '')
                tehaihinban2 = tehaihinban.replace(' ', '')
                if hinban3 == tehaihinban2:
                    day = pre_zaikodata.loc[i,'回収日時'].day
                    index = tehaidata.columns.get_loc("翌々々月稼働日数") + day
                    #count = pre_zaikodata.groupby('品番')['かんばんシリアル'].count()
                    #pre_zaikodata.loc[i,'実かんばん数']　= 
                    #print(day,index,str(tehaidata.iloc[j,index]),str(tehaidata.iloc[j,index]).replace(',', ''))
                    pre_zaikodata.loc[i,'日量数'] = int(str(tehaidata.iloc[j,index]).replace(',', ''))
                    pre_zaikodata.loc[i,'日量数（箱数）'] = pre_zaikodata.loc[i,'日量数'] /pre_zaikodata.loc[i, '収容数']
                    if int(str(tehaidata.loc[j,'当月必要数']).replace(',', '')) != 0:
                        avenichiryo = int(str(tehaidata.loc[j,'当月必要数']).replace(',', ''))/int(tehaidata.loc[j,'当月稼働日数'])
                        pre_zaikodata.loc[i,'平均日量数'] = avenichiryo
                        pre_zaikodata.loc[i,'平均日量数（箱数）'] = avenichiryo//pre_zaikodata.loc[i, '収容数']
                        pre_zaikodata.loc[i,'基準在庫枚数（日数）'] = (int(tehaidata.loc[j,'基準在庫枚数'])*pre_zaikodata.loc[i, '収容数'])/avenichiryo
                    else:
                        pre_zaikodata.loc[i,'平均日量数'] = 0
                        pre_zaikodata.loc[i,'平均日量数（箱数）'] = 0
                        pre_zaikodata.loc[i,'基準在庫枚数（日数）'] = 0
    else:
        pre_zaikodata.loc[i,'平均日量数'] = 0
        pre_zaikodata.loc[i,'平均日量数（箱数）'] = 0
        pre_zaikodata.loc[i,'基準在庫枚数（日数）'] = 0
print(count)

0
5000
15000
20000
25000
45000
0


In [6]:
count = 0
tehaidata = pd.read_csv('20231107使用_XR10_手配運用情報2300911_1Y（回収引当適用）.csv',encoding='cp932')
#tehaidata = pd.read_csv('sample2.csv',encoding='cp932')
long2 = tehaidata.iloc[:,1]

for i in range(len(long)):
    if (not pd.isnull(pre_zaikodata.loc[i, '発注〜印刷LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注〜検収LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注〜順立装置入庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注〜順立装置出庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注〜回収LT'])):
        hinban = pre_zaikodata.loc[i,'品番'] 
        hinban2 = hinban.replace("-", "") 
        hinban3 = hinban2.replace(" ", "") 
        if i % 5000==0:
            print(i,hinban2)
        for j in range(len(long2)):
            tehaihinban = tehaidata.loc[j,'品番'].replace('-', '')
            tehaihinban2 = tehaihinban.replace(' ', '')
            #print(tehaihinban)
            if hinban3 == tehaihinban2:
                pre_zaikodata.loc[i, 'A'] = tehaidata.loc[j,'納入ｻｲｸﾙ間隔']
                pre_zaikodata.loc[i, 'B'] = tehaidata.loc[j,'納入ｻｲｸﾙ回数']
                pre_zaikodata.loc[i, 'C'] = tehaidata.loc[j,'納入ｻｲｸﾙ情報']
                pre_zaikodata.loc[i, '便Ave'] = pre_zaikodata.loc[i,'日量数（箱数）']/pre_zaikodata.loc[i, 'B']
                pre_zaikodata.loc[i, '整備室'] = tehaidata.loc[j,'整備室']
                pre_zaikodata.loc[i, '箱種類'] = tehaidata.loc[j,'登録箱種']
                pre_zaikodata.loc[i, '箱サイズ'] = tehaidata.loc[j,'容積(m3)']
                pre_zaikodata.loc[i, '箱重量'] = tehaidata.loc[j,'総重量(Kg)']
                pre_zaikodata.loc[i, '基準在庫日数'] = tehaidata.loc[j,'基準在庫日数']
                pre_zaikodata.loc[i, '基準在庫枚数'] = tehaidata.loc[j,'基準在庫枚数']
                pre_zaikodata.loc[i, '設計かんばん数'] = pre_zaikodata.loc[i,'日量数（箱数）'] + pre_zaikodata.loc[i, '基準在庫枚数']
                pre_zaikodata.loc[i, '手配区分'] = tehaidata.loc[j,'手配区分']
                MAX= tehaidata.loc[j,'基準在庫日数'] + float(pre_zaikodata.loc[i,'基準在庫枚数（日数）'])
                pre_zaikodata.loc[i,'基準在庫日数（基準在庫枚数込み）'] = MAX
                zisseki = pre_zaikodata.loc[i, '社内LT（検収〜回収LT）'] 
                if  (zisseki > MAX*0)and(zisseki < 0.25*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常？：設計値の0〜0.25倍'
                elif  (zisseki > MAX*0.25)and(zisseki < 0.5*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常？：設計値の0.25〜0.5倍'
                elif  (zisseki > MAX*0.5)and(zisseki < 0.75*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常よりの異常？：設計値の0.5〜0.75倍'
                elif  (zisseki > MAX*0.75)and(zisseki < MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常：設計値の0.75〜1倍'
                elif (zisseki >= MAX)and(zisseki < MAX*1.25):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常：設計値の1〜1.25倍'
                elif (zisseki >= MAX*1.25)and(zisseki < 1.5*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常よりの異常：設計値の1.25〜1.5倍'
                elif (zisseki >= MAX*1.5)and(zisseki < 2*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常：設計値の1.5〜2倍'
                elif (zisseki >= MAX*2):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常：設計値の2倍以上'
        #print(LT2,LT3,LT4,LT5)
        count = count + 1
print(count)

0 G1144ECE010    
5000 9036340A010    
10000 G9210ECE020    
15000 G1144ECE010    
20000 G1144ECB010    
25000 41310ECE010    
30000 35847ECE010    
35000 G9210ECE020    
40000 39171ECB010    
45000 35771ECE010    
50000 35771ECB010    
55000 41310ECE010    
58113


In [44]:
# 新しいCSVファイルのパスを定義します。
new_file_path = '20231107使用_在庫推移9月.csv'

# 'cp932'エンコーディングを使用してCSVファイルを読み込みます。
new_kanban_data = pd.read_csv(new_file_path, encoding='cp932')

# '計測日時'を日時型に変換し、週番号を含む新しい列を追加します。
new_kanban_data['計測日時'] = pd.to_datetime(new_kanban_data['計測日時'], errors='coerce')
new_kanban_data.dropna(subset=['計測日時'], inplace=True)
new_kanban_data['Year'] = new_kanban_data['計測日時'].dt.year
new_kanban_data['Week_Number'] = new_kanban_data['計測日時'].dt.week

# '品番'と'Week_Number'でグループ化し、'在庫数（箱）', '入庫数（箱）', '出庫数（箱）'を合計します。
weekly_inventory_aggregated = new_kanban_data.groupby(['品番', 'Year', 'Week_Number']).agg({
    '在庫数（箱）': 'median',
    '入庫数（箱）': 'median',
    '出庫数（箱）': 'median'
}).reset_index()

# 集計したデータを新しいCSVファイルに保存します。
with open('kari1.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    weekly_inventory_aggregated.to_csv(f)

weekly_inventory_aggregated.head(300)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）
0,019120LC030,2023,36,12.0,0.0,0.0
1,019120LC030,2023,37,11.5,0.0,0.0
2,019120LC030,2023,38,11.0,0.0,0.0
3,019120LC030,2023,39,11.0,0.0,0.0
4,019120LC040,2023,36,3.0,0.0,0.0
...,...,...,...,...,...,...
295,35409ECB010,2023,39,2.5,0.0,0.0
296,354110LC030,2023,36,44.0,0.0,0.0
297,354110LC030,2023,37,41.0,0.0,0.0
298,354110LC030,2023,38,42.5,0.0,0.0


In [45]:
# '納入日'を日時型に変換
pre_zaikodata['納入日'] = pd.to_datetime(pre_zaikodata['納入日'], errors='coerce')
# 日時型に変換できなかった行を削除
pre_zaikodata.dropna(subset=['納入日'], inplace=True)

# 'Week_Number' 列を追加（年と週の番号を取得）
pre_zaikodata['Year'] = pre_zaikodata['納入日'].dt.year
pre_zaikodata['Week_Number'] = pre_zaikodata['納入日'].dt.week

# '品番'と'Week_Number'でグループ化し、'収容数'を合計
weekly_aggregated = pre_zaikodata.groupby(['品番', '仕入先名','Year', 'Week_Number', '基準在庫日数', '基準在庫枚数','箱種類','A','B','C']).agg({'収容数': 'median', '納入便': 'median', '社内LT（検収〜回収LT）': 'median','日量数（箱数）':'median'})
# '品番'と'Week_Number'でグループ化し、各グループの行数（かんばん数）をカウント
weekly_kanban_count = pre_zaikodata.groupby(['品番','仕入先名','Year', 'Week_Number', '基準在庫日数', '基準在庫枚数','箱種類','A','B','C']).size().reset_index(name='かんばん数')

# 2つの集計データセットを結合
merged_weekly_data = pd.merge(weekly_aggregated, weekly_kanban_count, on=['品番', 'Year', 'Week_Number', '基準在庫日数', '基準在庫枚数','箱種類','A','B','C'], how='outer')

# 集計したデータをCSVファイルに保存
with open('kari2.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_weekly_data.to_csv(f)

merged_weekly_data.head(30)

,品番,Year,Week_Number,基準在庫日数,基準在庫枚数,箱種類,A,B,C,収容数,納入便,社内LT（検収〜回収LT）,日量数（箱数）,仕入先名,かんばん数
0,019128GA010,2023,35,1.07,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50,2.0,3.320,18.900000,石黒ゴム工業（株）,91
1,019128GA010,2023,36,1.07,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50,2.0,3.860,18.900000,石黒ゴム工業（株）,85
2,019128GA010,2023,37,1.07,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50,1.5,3.530,17.540000,石黒ゴム工業（株）,92
3,019128GA010,2023,38,1.07,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50,2.0,3.425,17.540000,石黒ゴム工業（株）,84
4,019128GA010,2023,39,1.07,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50,1.0,2.255,16.440000,石黒ゴム工業（株）,52
5,01912ECB010,2023,35,0.85,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300,2.0,4.620,6.306667,石黒ゴム工業（株）,29
6,01912ECB010,2023,36,0.85,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300,1.0,3.520,6.300000,石黒ゴム工業（株）,31
7,01912ECB010,2023,37,0.85,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300,1.0,3.250,5.846667,石黒ゴム工業（株）,30
8,01912ECB010,2023,38,0.85,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300,1.5,3.625,5.846667,石黒ゴム工業（株）,30
9,01912ECB010,2023,39,0.85,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300,2.0,2.875,5.480000,石黒ゴム工業（株）,16


In [46]:
# 2つの集計データセットを結合
merged_data = pd.merge(weekly_inventory_aggregated, merged_weekly_data, on=['品番', 'Year', 'Week_Number'], how='inner')
# '基準在庫日数' から '社内LT（検収～回収LT）' を引いた新しい列 'LT_差分' を追加
merged_data['実績LT/設計値LT'] = merged_data['社内LT（検収〜回収LT）']/merged_data['基準在庫日数']
merged_data['順立装置在庫量/設計値MIN']=merged_data['在庫数（箱）']/(0.1*(merged_data['日量数（箱数）']*merged_data['A']*(1+merged_data['C'])/merged_data['B']))

# Load the new CSV file into a DataFrame with 'Shift_JIS' encoding
pitch = pd.read_csv('不等ピッチ情報.csv', encoding='Shift_JIS')

longi = len(merged_data.loc[:,"品番"])
longj = len(pitch.loc[:,"仕入先名"])

for i in range(longi):
    for j in range(longj):
        if (merged_data.loc[i, '仕入先名'] == pitch.loc[j,'仕入先名']):
            merged_data.loc[i,"不等ピッチ"] = pitch.loc[j,"不等ピッチ係数日"]
            
merged_data['順立装置在庫量/設計値MAX'] = merged_data['在庫数（箱）']/(merged_data['順立装置在庫量/設計値MIN'] + merged_data['日量数（箱数）']/merged_data['B'] + merged_data['日量数（箱数）']*merged_data.loc[i,"不等ピッチ"])

#S = int(str(df_design.loc[j,'収容数']).replace(',', ''))
#df_merge.loc[count,'最小値'] = 0.1 * ((daily/S)  * A * (1 + C) / B) 
#df_merge.loc[count,'最大値'] = df_merge.loc[count,'最小値'] + (daily / S)/B + (daily/S)*df_design.loc[j,'不等ピッチ']

# 集計したデータをCSVファイルに保存
with open('kari5.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_data.to_csv(f)

merged_data.head(300)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫枚数,箱種類,A,...,収容数,納入便,社内LT（検収〜回収LT）,日量数（箱数）,仕入先名,かんばん数,実績LT/設計値LT,順立装置在庫量/設計値MIN,不等ピッチ,順立装置在庫量/設計値MAX
0,019128GA010,2023,36,41.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,50,2.0,3.860,18.900000,石黒ゴム工業（株）,85,3.607477,9.350483,0.29,2.076424
1,019128GA010,2023,37,40.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,50,1.5,3.530,17.540000,石黒ゴム工業（株）,92,3.299065,9.829749,0.29,2.053731
2,019128GA010,2023,38,37.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,50,2.0,3.425,17.540000,石黒ゴム工業（株）,84,3.200935,9.092518,0.29,1.974437
3,019128GA010,2023,39,40.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,50,1.0,2.255,16.440000,石黒ゴム工業（株）,52,2.107477,10.487457,0.29,2.048188
4,01912ECB010,2023,36,13.0,0.0,0.0,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,300,1.0,3.520,6.300000,石黒ゴム工業（株）,31,4.141176,8.894362,0.29,1.051834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,9011906A065,2023,38,2.0,0.0,0.0,0.53,1.0,ＴＰ３３１フタナシ,1.0,...,1000,4.0,1.565,2.631000,（株）青山製作所,14,2.952830,6.747046,0.08,0.273332
296,9011906A065,2023,39,3.0,0.0,0.0,0.53,1.0,ＴＰ３３１フタナシ,1.0,...,1000,2.5,1.565,2.634000,（株）青山製作所,8,2.952830,10.109043,0.08,0.280906
297,9014908A008,2023,36,5.0,0.0,0.0,0.52,1.0,ＴＰ３３１フタナシ,1.0,...,800,2.0,1.735,3.543750,（株）青山製作所,23,3.336538,12.523089,0.08,0.376197
298,9014908A008,2023,37,4.0,0.0,0.0,0.52,1.0,ＴＰ３３１フタナシ,1.0,...,800,4.0,1.760,3.230625,（株）青山製作所,13,3.384615,10.989502,0.08,0.342188


In [48]:
# 新しいCSVファイルのパスを定義します。
new_file_path = '20231107使用_組立稼働率実績.csv'

# 'cp932'エンコーディングを使用してCSVファイルを読み込みます。
AVAILABLE_RATE = pd.read_csv(new_file_path, encoding='cp932')

# データフレームのインデックスを日付型に変換
AVAILABLE_RATE['LINE_DATE'] = pd.to_datetime(AVAILABLE_RATE['LINE_DATE'])
#AVAILABLE_RATE.set_index('LINE_DATE', inplace=True)
# 'WEEK_NUMBER'列を追加し、'LINE_DATE'の週番号を計算
AVAILABLE_RATE['Week_Number'] = AVAILABLE_RATE['LINE_DATE'].dt.week
AVAILABLE_RATE['WEEKDAY'] = AVAILABLE_RATE['LINE_DATE'].dt.weekday

# 土日を除外
AVAILABLE_RATE2 = AVAILABLE_RATE[(AVAILABLE_RATE['WEEKDAY'] != 5) & (AVAILABLE_RATE['WEEKDAY'] != 6)]

# 'AVAILABLE_RATE'列を週単位でリサンプリングし、平均を計算
weekly_average_available_rate = AVAILABLE_RATE2.groupby('Week_Number').agg({'OPERATION_RATE': 'median', 'KAKO_CNT': 'median'})
#['AVAILABLE_RATE'].mean().reset_index()
#weekly_average_available_rate = AVAILABLE_RATE2.groupby('Week_Number')['KAKO_CNT'].mean().reset_index()

# 集計したデータを新しいCSVファイルに保存します。
with open('kari4.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    weekly_average_available_rate.to_csv(f)

weekly_average_available_rate.head(300)

,OPERATION_RATE,KAKO_CNT
Week_Number,,
35,91.025,424.5
36,97.810,480.0
37,93.505,462.0
38,97.085,450.0
39,99.450,464.0


In [49]:
# 2つの集計データセットを結合
merged_data = pd.merge(merged_data, weekly_average_available_rate, on=['Week_Number'], how='inner')
# 集計したデータを新しいCSVファイルに保存します。
with open('kari6.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_data.to_csv(f)
merged_data.head(300)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫枚数,箱種類,A,...,社内LT（検収〜回収LT）,日量数（箱数）,仕入先名,かんばん数,実績LT/設計値LT,順立装置在庫量/設計値MIN,不等ピッチ,順立装置在庫量/設計値MAX,OPERATION_RATE,KAKO_CNT
0,019128GA010,2023,36,41.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,3.860,18.900000,石黒ゴム工業（株）,85,3.607477,9.350483,0.29,2.076424,97.810,480.0
1,01912ECB010,2023,36,13.0,0.0,0.0,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,3.520,6.300000,石黒ゴム工業（株）,31,4.141176,8.894362,0.29,1.051834,97.810,480.0
2,01912ECB040,2023,36,30.5,0.0,1.0,0.25,1.0,ＴＰ３４２フタナシ,1.0,...,2.335,18.900000,愛産樹脂工業（株）,86,9.340000,5.172297,0.00,1.219156,97.810,480.0
3,01912ECB060,2023,36,3.0,0.0,0.0,0.78,1.0,ＴＰ３３１フタナシ,1.0,...,6.145,1.028750,新光ゴム工業（株）,4,7.878205,6.861554,0.00,0.377751,97.810,480.0
4,1040043104R,2023,36,4.0,0.0,0.0,0.75,3.0,AW131ﾌﾀﾅｼ,1.0,...,5.260,1.890000,（株）ギフ加藤製作所,8,7.013333,8.818342,0.12,0.405768,97.810,480.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,G1144ECE010,2023,37,61.0,0.0,2.0,0.58,2.0,ＴＰ３３１フタアリ,1.0,...,1.600,62.375000,矢崎総業（株）,326,2.758621,3.505218,0.00,0.884071,93.505,462.0
296,G1163ECB010,2023,37,3.0,0.0,0.0,0.54,1.0,ＴＰ３３１フタナシ,1.0,...,0.730,8.775000,（株）青山製作所,44,1.351852,3.034441,0.08,0.607818,93.505,462.0
297,G1163ECE010,2023,37,11.0,0.0,1.0,0.53,1.0,ＴＰ３３１フタナシ,1.0,...,0.960,19.290000,（株）青山製作所,106,1.811321,5.061334,0.08,1.190369,93.505,462.0
298,G117362010,2023,37,54.0,0.0,2.0,0.85,1.0,ＴＰ３３１．５フタナシ,1.0,...,1.890,48.722222,多摩川精機販売（株）,260,2.223529,2.736602,0.00,1.001949,93.505,462.0


In [54]:
# 品番と週番号でデータを並べ替え
merged_data2 = merged_data.sort_values(by=['品番', 'Week_Number'])
# 同じ品番ごとに在庫数の週間差分を計算
merged_data2['先週からの順立装置在庫量増加率'] = merged_data2.groupby('品番')['在庫数（箱）'].pct_change()
merged_data2['先週からの社内LT増加率'] = merged_data2.groupby('品番')['社内LT（検収〜回収LT）'].pct_change()
merged_data2['先週からのかんばん増加率'] = merged_data2.groupby('品番')['かんばん数'].pct_change()
# 差分がNaN（最初の週）である場合は0に置き換え
merged_data2['先週からの順立装置在庫量増加率'].fillna(0, inplace=True)
merged_data2['先週からの社内LT増加率'].fillna(0, inplace=True)
merged_data2['先週からのかんばん増加率'].fillna(0, inplace=True)
merged_data2.head(30)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫枚数,箱種類,A,...,かんばん数,実績LT/設計値LT,順立装置在庫量/設計値MIN,不等ピッチ,順立装置在庫量/設計値MAX,OPERATION_RATE,KAKO_CNT,先週からの順立装置在庫量増加率,先週からの社内LT増加率,先週からのかんばん増加率
0,019128GA010,2023,36,41.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,85,3.607477,9.350483,0.29,2.076424,97.810,480.0,0.000000,0.000000,0.000000
151,019128GA010,2023,37,40.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,92,3.299065,9.829749,0.29,2.053731,93.505,462.0,-0.024390,-0.085492,0.082353
306,019128GA010,2023,38,37.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,84,3.200935,9.092518,0.29,1.974437,97.085,450.0,-0.075000,-0.029745,-0.086957
450,019128GA010,2023,39,40.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,52,2.107477,10.487457,0.29,2.048188,99.450,464.0,0.081081,-0.341606,-0.380952
1,01912ECB010,2023,36,13.0,0.0,0.0,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,31,4.141176,8.894362,0.29,1.051834,97.810,480.0,0.000000,0.000000,0.000000
152,01912ECB010,2023,37,13.0,0.0,0.0,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,30,3.823529,9.584005,0.29,1.015651,93.505,462.0,0.000000,-0.076705,-0.032258
307,01912ECB010,2023,38,12.0,0.0,0.0,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,30,4.264706,8.846774,0.29,0.994824,97.085,450.0,-0.076923,0.115385,0.000000
451,01912ECB010,2023,39,14.0,0.0,0.0,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,16,3.382353,11.011830,0.29,0.998158,99.450,464.0,0.166667,-0.206897,-0.466667
2,01912ECB040,2023,36,30.5,0.0,1.0,0.25,1.0,ＴＰ３４２フタナシ,1.0,...,86,9.340000,5.172297,0.00,1.219156,97.810,480.0,0.000000,0.000000,0.000000
153,01912ECB040,2023,37,26.0,0.0,1.0,0.25,1.0,ＴＰ３４２フタナシ,1.0,...,93,6.120000,4.751045,0.00,1.122235,93.505,462.0,-0.147541,-0.344754,0.081395


In [55]:
# 新しいCSVファイルのパスを定義します。
new_file_path = '20231107使用_使用工程.csv'
# 'cp932'エンコーディングを使用してCSVファイルを読み込みます。
step = pd.read_csv(new_file_path, encoding='cp932')

longi = len(merged_data2.loc[:,"品番"])
longj = len(step.loc[:,"品番"])

for i in range(longi):
    hinban = merged_data2.loc[i,'品番'] 
    hinban2 = hinban.replace("-", "") 
    hinban3 = hinban2.replace(" ", "") 
    for j in range(longj):
        stephinban = step.loc[j,'品番'].replace('-', '')
        stephinban2 = stephinban.replace(' ', '')
        if (hinban3 == stephinban2):
            merged_data2.loc[i,"使用工程"] = step.loc[j,"使用工程"]
    
merged_data2.head(300)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫枚数,箱種類,A,...,実績LT/設計値LT,順立装置在庫量/設計値MIN,不等ピッチ,順立装置在庫量/設計値MAX,OPERATION_RATE,KAKO_CNT,先週からの順立装置在庫量増加率,先週からの社内LT増加率,先週からのかんばん増加率,使用工程
0,019128GA010,2023,36,41.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,3.607477,9.350483,0.29,2.076424,97.810,480.0,0.000000,0.000000,0.000000,8.ファイナル後半
151,019128GA010,2023,37,40.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,3.299065,9.829749,0.29,2.053731,93.505,462.0,-0.024390,-0.085492,0.082353,8.ファイナル後半
306,019128GA010,2023,38,37.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,3.200935,9.092518,0.29,1.974437,97.085,450.0,-0.075000,-0.029745,-0.086957,8.ファイナル後半
450,019128GA010,2023,39,40.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,2.107477,10.487457,0.29,2.048188,99.450,464.0,0.081081,-0.341606,-0.380952,8.ファイナル後半
1,01912ECB010,2023,36,13.0,0.0,0.0,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,4.141176,8.894362,0.29,1.051834,97.810,480.0,0.000000,0.000000,0.000000,8.ファイナル後半
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,9011906A065,2023,38,2.0,0.0,0.0,0.53,1.0,ＴＰ３３１フタナシ,1.0,...,2.952830,6.747046,0.08,0.273332,97.085,450.0,0.000000,-0.158602,0.000000,2.MGケース
522,9011906A065,2023,39,3.0,0.0,0.0,0.53,1.0,ＴＰ３３１フタナシ,1.0,...,2.952830,10.109043,0.08,0.280906,99.450,464.0,0.500000,0.000000,-0.428571,2.MGケース
75,9014908A008,2023,36,5.0,0.0,0.0,0.52,1.0,ＴＰ３３１フタナシ,1.0,...,3.336538,12.523089,0.08,0.376197,97.810,480.0,0.000000,0.000000,0.000000,7.ファイナル前半
226,9014908A008,2023,37,4.0,0.0,0.0,0.52,1.0,ＴＰ３３１フタナシ,1.0,...,3.384615,10.989502,0.08,0.342188,93.505,462.0,-0.200000,0.014409,-0.434783,7.ファイナル前半


In [56]:
# Week_Numberが36のデータを削除
merged_data2 = merged_data2[merged_data2['Week_Number'] != 36]

# 集計したデータを新しいCSVファイルに保存します。
with open('data.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_data2.to_csv(f)

# 確認のため、削除後のデータの最初の5行を表示
merged_data2.head()

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫枚数,箱種類,A,...,実績LT/設計値LT,順立装置在庫量/設計値MIN,不等ピッチ,順立装置在庫量/設計値MAX,OPERATION_RATE,KAKO_CNT,先週からの順立装置在庫量増加率,先週からの社内LT増加率,先週からのかんばん増加率,使用工程
151,019128GA010,2023,37,40.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,3.299065,9.829749,0.29,2.053731,93.505,462.0,-0.024390,-0.085492,0.082353,8.ファイナル後半
306,019128GA010,2023,38,37.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,3.200935,9.092518,0.29,1.974437,97.085,450.0,-0.075000,-0.029745,-0.086957,8.ファイナル後半
450,019128GA010,2023,39,40.0,0.0,1.0,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,2.107477,10.487457,0.29,2.048188,99.450,464.0,0.081081,-0.341606,-0.380952,8.ファイナル後半
152,01912ECB010,2023,37,13.0,0.0,0.0,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,3.823529,9.584005,0.29,1.015651,93.505,462.0,0.000000,-0.076705,-0.032258,8.ファイナル後半
307,01912ECB010,2023,38,12.0,0.0,0.0,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,4.264706,8.846774,0.29,0.994824,97.085,450.0,-0.076923,0.115385,0.000000,8.ファイナル後半


In [57]:
step.head(300)

,品番,使用工程
0,34989ECE030,NaN
1,9036340A010,NaN
2,35145ECE020,2.MGケース
3,33490ECB010,7.ファイナル前半
4,33490ECE010,7.ファイナル前半
...,...,...
143,9036652A004,1.デフ・大物ギヤ・Brg
144,9036628A007,1.デフ・大物ギヤ・Brg
145,9036630A017,1.デフ・大物ギヤ・Brg
146,9036628A006,1.デフ・大物ギヤ・Brg
